In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import sklearn as skl

In [2]:
func = lambda a,b,c: 5*a + 3*b + 1.5*c + 6 

In [3]:
# Generate data
X = np.random.randint(0, 10, size=(1000, 3))

In [4]:
# Generate labels
Y = np.array([func(*x) for x in X])

In [5]:
X = np.append(X, np.ones((1000, 1)), axis=1)

In [6]:
D = np.append(X, Y.reshape(-1, 1), axis=1)

In [7]:
TestSplit, TrainSplit = ms.train_test_split(D, test_size=0.2)

In [8]:
def closedForm(X,Y):
    XTX = np.matmul(X.T, X)
    XTX_inv = np.linalg.inv(XTX)
    XTX_inv_XT = np.matmul(XTX_inv, X.T)

    W = np.matmul(XTX_inv_XT, Y)
    return W

In [9]:
Weights = closedForm(TrainSplit[:,:-1], TrainSplit[:,-1])

In [10]:
def accuracy(W, X, Y):
    return (np.matmul((np.matmul(X, W) - Y).T, (np.matmul(X, W) - Y))* 1/2)**0.5


Accuracy on Training Data

In [11]:
accuracy(Weights, X, Y)

9.766433488670575e-13

Accuracy on Test Data

In [12]:
accuracy(Weights, TestSplit[:,:-1], TestSplit[:,-1])

8.791925599532235e-13

In [13]:
df = pd.read_csv('USA_Housing.csv')
df.dropna(inplace=True)
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.45857,5.682861,7.009188,4.09,23086.80050,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.64245,6.002900,6.730821,3.09,40173.07217,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.06718,5.865890,8.512727,5.13,36882.15940,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.24005,7.188236,5.586729,3.26,34310.24283,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.19723,5.040555,7.839388,4.23,26354.10947,6.309435e+05,USNS Raymond\nFPO AE 09386


In [14]:
TrainSplit2, TestSplit2 = ms.train_test_split(df, test_size=0.2)


In [15]:
def splitFL(df):
    X = df.drop(['Price', 'Address'], axis=1)
    Y = df['Price']
    return np.array(X), np.array(Y)

In [16]:
XHT, YHT = splitFL(TrainSplit2)

In [17]:
## regularize data
def regularize(X):
    return (X - X.mean(axis=0))/X.std(axis=0)

In [18]:
def Dloss(Y, w, X):
    n = X.shape[0]
    summ = 0
    for i in range (n):
        pred = np.dot(w,X[i])
        diff = pred - Y[i]
        diffx = diff * X[i]
        summ += diffx
    
    return summ/n

In [19]:
def DlossMat(Y, w, X):
    n = X.shape[0]
    return np.matmul(X.T, np.matmul(X, w) - Y)/n

In [20]:
def gradient_descent(X, Y, w, iterations):
    alpha = 1
    for i in range(iterations):
        w = w - alpha * Dloss(Y, w, X)
    return w

In [21]:
def gd_mat(X, Y, w, iterations):
    alpha = 1
    for i in range(iterations):
        w = w - alpha * DlossMat(Y, w, X)
    return w

In [22]:
def prepare(X, Y):
    X = regularize(X)
    Y = regularize(Y)
    X = np.append(X, np.ones((X.shape[0], 1)), axis=1)
    w = np.zeros(X.shape[1])
    return X, Y, w

In [23]:
Xin, Yin, Win = prepare(XHT, YHT)
Wgd = gradient_descent(Xin, Yin, Win, 100)

In [33]:
Agd = accuracy(Wgd, Xin, Yin)
print(Agd)

12.799498867585106


In [34]:
AgdMat = accuracy(gd_mat(Xin, Yin, Win, 100), Xin, Yin)
print(AgdMat)

12.799498867585106


Gradient Descent Vs Closed Form on Training Data

In [36]:
Wcf = closedForm(Xin, Yin)
Acf = accuracy(Wcf, Xin, Yin)
AccuracyDiff = Acf - Agd
AccuracyDiff

0.0

Gradient Descent Accuracy on Test Split

In [35]:
XTest , YTest = splitFL(TestSplit2)
XTest, YTest, WTest = prepare(XTest, YTest)
print(accuracy(Wgd, XTest, YTest))

6.45661431978366


Closed Form Solution Accuracy on Test Split

In [32]:
print(accuracy(Wcf, XTest, YTest))

6.45661431978366
